In [1]:
import pandas as pd
import numpy as np

# Load dữ liệu
df = pd.read_csv('../data/merged_dataset.csv')

# Xem thông tin cơ bản
print(f"Số lượng dòng: {len(df)}")
print(f"\nCác cột: {df.columns.tolist()}")
print(f"\nThông tin chi tiết:")
print(df.info())

Số lượng dòng: 164069

Các cột: ['text', 'label', 'source', 'language']

Thông tin chi tiết:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164069 entries, 0 to 164068
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      164069 non-null  object
 1   label     164069 non-null  int64 
 2   source    164069 non-null  object
 3   language  164069 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.0+ MB
None


In [2]:
# Xem 5 dòng đầu
print(df.head())

# Xem ngẫu nhiên 5 dòng
print(df.sample(5))

# Xem riêng dữ liệu tiếng Việt
vietnamese_df = df[df['language'] == 'vi']
print(f"\nSố lượng comments tiếng Việt: {len(vietnamese_df)}")
print(vietnamese_df.sample(3))

                                                text  label  source language
0  Explanation\nWhy the edits made under my usern...      0  jigsaw       en
1  D'aww! He matches this background colour I'm s...      0  jigsaw       en
2  Hey man, I'm really not trying to edit war. It...      0  jigsaw       en
3  "\nMore\nI can't make any real suggestions on ...      0  jigsaw       en
4  You, sir, are my hero. Any chance you remember...      0  jigsaw       en
                                                     text  label  source  \
47398   "\n\nEach in turn\n\nSuggested: In physics, fo...      0  jigsaw   
103425                 REDIRECT Talk:Pedro Daniel Estrada      0  jigsaw   
127904  Why to go buddy! So how is Nancy holding up?\n...      1  jigsaw   
70185   "\n No, not phosphorescent. Phosphorescence, a...      0  jigsaw   
36426   HELP \n\nHEY BUDDY I DON'T LIKE AT ALL!!!  \n\...      0  jigsaw   

       language  
47398        en  
103425       en  
127904       en  
70185    

In [3]:
# Kiểm tra null
print("Số lượng null trong mỗi cột:")
print(df.isnull().sum())

# Kiểm tra chi tiết text null
null_texts = df[df['text'].isnull()]
print(f"\nSố dòng có text null: {len(null_texts)}")

Số lượng null trong mỗi cột:
text        0
label       0
source      0
language    0
dtype: int64

Số dòng có text null: 0


In [4]:
# Kiểm tra text rỗng (chỉ có khoảng trắng)
empty_texts = df[df['text'].str.strip() == '']
print(f"Số comments trống: {len(empty_texts)}")

# Loại bỏ
df = df[df['text'].str.strip() != '']
print(f"Sau khi loại text trống: {len(df)} dòng")

Số comments trống: 0
Sau khi loại text trống: 164069 dòng


In [5]:
# Xem phân bố độ dài
df['text_length'] = df['text'].str.len()
print(f"\nĐộ dài ngắn nhất: {df['text_length'].min()}")
print(f"Độ dài dài nhất: {df['text_length'].max()}")
print(f"Độ dài trung bình: {df['text_length'].mean():.1f}")

# Xem những comments quá ngắn
very_short = df[df['text_length'] < 5]
print(f"\nSố comments < 5 ký tự: {len(very_short)}")
print("\nVí dụ:")
print(very_short['text'].head(10))


Độ dài ngắn nhất: 6
Độ dài dài nhất: 5000
Độ dài trung bình: 385.9

Số comments < 5 ký tự: 0

Ví dụ:
Series([], Name: text, dtype: object)


In [6]:
# Quyết định threshold (ví dụ: >= 10 ký tự)
MIN_LENGTH = 10
df = df[df['text_length'] >= MIN_LENGTH]
print(f"Sau khi loại comments < {MIN_LENGTH} ký tự: {len(df)} dòng")

Sau khi loại comments < 10 ký tự: 164064 dòng


In [7]:
# Kiểm tra duplicates
duplicates = df.duplicated(subset=['text'])
print(f"Số comments bị duplicate: {duplicates.sum()}")

# Xem ví dụ
if duplicates.sum() > 0:
    dup_examples = df[df.duplicated(subset=['text'], keep=False)].sort_values('text')
    print("\nVí dụ duplicates:")
    print(dup_examples[['text', 'label', 'source']].head(10))

Số comments bị duplicate: 0


In [8]:
# Reset index sau khi đã xóa nhiều dòng
df = df.reset_index(drop=True)
print(f"\nTổng số dòng cuối cùng: {len(df)}")


Tổng số dòng cuối cùng: 164064


In [9]:
# Xem trước và sau
print("Trước khi lowercase:")
print(df['text'].iloc[0])

df['text'] = df['text'].str.lower()

print("\nSau khi lowercase:")
print(df['text'].iloc[0])

Trước khi lowercase:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Sau khi lowercase:
explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27


In [10]:
import re

# Hàm loại bỏ URLs
def remove_urls(text):
    # Pattern cho URLs
    url_pattern = r'http\S+|www\.\S+|https\S+'
    text = re.sub(url_pattern, '', text)
    return text

# Test trước
test_text = "xem video này http://youtube.com/abc hay lắm"
print(f"Trước: {test_text}")
print(f"Sau: {remove_urls(test_text)}")

Trước: xem video này http://youtube.com/abc hay lắm
Sau: xem video này  hay lắm


In [11]:
# Chỉ áp dụng cho tiếng Việt
vietnamese_mask = df['language'] == 'vi'
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(remove_urls)

In [12]:
def remove_mentions(text):
    # Pattern: @ theo sau bởi ký tự chữ/số
    mention_pattern = r'@\w+'
    text = re.sub(mention_pattern, '', text)
    return text

# Test
test_text = "@admin xin chào @user123 bạn ơi"
print(f"Trước: {test_text}")
print(f"Sau: {remove_mentions(test_text)}")

Trước: @admin xin chào @user123 bạn ơi
Sau:  xin chào  bạn ơi


In [13]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(remove_mentions)

In [14]:
def process_hashtags(text):
    # Loại bỏ # nhưng GIỮ text
    # #toxic → toxic
    hashtag_pattern = r'#(\w+)'
    text = re.sub(hashtag_pattern, r'\1', text)
    return text

# Test
test_text = "bài này #toxic #spam quá"
print(f"Trước: {test_text}")
print(f"Sau: {process_hashtags(test_text)}")

Trước: bài này #toxic #spam quá
Sau: bài này toxic spam quá


In [15]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(process_hashtags)

In [16]:
def remove_emails(text):
    email_pattern = r'\S+@\S+\.\S+'
    text = re.sub(email_pattern, '', text)
    return text

# Test
test_text = "liên hệ tôi qua email abc@gmail.com nhé"
print(f"Trước: {test_text}")
print(f"Sau: {remove_emails(test_text)}")

Trước: liên hệ tôi qua email abc@gmail.com nhé
Sau: liên hệ tôi qua email  nhé


In [17]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(remove_emails)

In [18]:
def normalize_whitespace(text):
    # Thay nhiều spaces bằng 1 space
    text = re.sub(r'\s+', ' ', text)
    # Loại bỏ space đầu/cuối
    text = text.strip()
    return text

# Test
test_text = "text   này    có     nhiều      spaces"
print(f"Trước: '{test_text}'")
print(f"Sau: '{normalize_whitespace(test_text)}'")

Trước: 'text   này    có     nhiều      spaces'
Sau: 'text này có nhiều spaces'


In [19]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(normalize_whitespace)

In [20]:
def remove_repeated_chars(text):
    # "hayyyyyyy" → "hayy" (giữ tối đa 2 ký tự lặp)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text

# Test
test_text = "waaaaaa đẹpppppp quáááááá"
print(f"Trước: {test_text}")
print(f"Sau: {remove_repeated_chars(test_text)}")

Trước: waaaaaa đẹpppppp quáááááá
Sau: waa đẹpp quáá


In [21]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(remove_repeated_chars)

In [22]:
import emoji

def remove_emoji_v2(text):
    """
    Loại bỏ emoji sử dụng thư viện emoji
    Phương pháp này toàn diện hơn regex
    """
    # Thay thế tất cả emoji bằng chuỗi rỗng
    return emoji.replace_emoji(text, replace='')


In [23]:
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(remove_emoji_v2)

Xử lý teecode


In [24]:
teencode_dict = {
    # Phủ định
    'ko': 'không', 'k': 'không', 'khong': 'không', 'hok': 'không',
    'kh': 'không', 'hong': 'không', 'hông': 'không',
    
    # Đồng ý
    'ok': 'được', 'oke': 'được', 'okie': 'được', 'okela': 'được',
    'dc': 'được', 'đc': 'được',
    
    # Đại từ
    'mik': 'mình', 'mk': 'mình', 'mjk': 'mình',
    'vs': 'với', 'v': 'vậy', 'z': 'vậy', 'vay': 'vậy',
    
    # Từ thông dụng
    'bik': 'biết', 'bit': 'biết', 'biet': 'biết',
    'wa': 'quá', 'wá': 'quá', 'qua': 'quá',
    'j': 'gì', 'gi': 'gì',
    'lm': 'làm', 'lam': 'làm',
    'ms': 'mới', 'moi': 'mới',
    'r': 'rồi', 'rui': 'rồi', 'ròi': 'rồi',
    'cx': 'cũng', 'cug': 'cũng',
    'th': 'thì', 'thi': 'thì',
    
    # Toxic (GIỮ CẢ HAI - quan trọng!)
    'vcl': 'vãi cả lồn',
    'vl': 'vãi lồn',
    'dm': 'địt mẹ',
    'dmm': 'địt mẹ mày',
    'đm': 'địt mẹ',
    'cc': 'cặc',
    'dcm': 'địt con mẹ',
    'đcm': 'địt con mẹ',
    'clm': 'cái lồn mẹ',
    'cmm': 'con mẹ mày',
    'cmn': 'con mẹ nó',
    'đ': 'đéo',
    'wtf': 'what_the_fuck',
    'loz': 'lồn'
}

In [25]:
def expand_teencode(text, teencode_dict):
    """
    Mở rộng teencode - GIỮ cả gốc và full
    Ví dụ: "ngu vcl" → "ngu vcl vãi_cái_lồn"
    """
    words = text.split()
    expanded_words = []
    
    for word in words:
        # Luôn giữ từ gốc
        expanded_words.append(word)
        
        # Nếu là teencode, thêm từ đầy đủ
        if word in teencode_dict:
            full_word = teencode_dict[word]
            expanded_words.append(full_word)
    
    return ' '.join(expanded_words)
# Test
test_text = "thằng ngu vcl dm"
result = expand_teencode(test_text, teencode_dict)
print(f"Trước: {test_text}")
print(f"Sau: {result}")


Trước: thằng ngu vcl dm
Sau: thằng ngu vcl vãi cả lồn dm địt mẹ


In [26]:
vietnamese_mask = df['language'] == 'vi'
df.loc[vietnamese_mask, 'text'] = df.loc[vietnamese_mask, 'text'].apply(
    lambda x: expand_teencode(x, teencode_dict)
)

print("✅ Đã expand teencode")

✅ Đã expand teencode


In [27]:


print("="*60)
print("KIỂM TRA VÀ XỬ LÝ DỮ LIỆU (ĐÃ SỬA LỖI)")
print("="*60)

# Bắt đầu với df từ cell trước (164,064 dòng)
print(f"Tổng số documents ban đầu: {len(df):,}")

# 1. Kiểm tra null/NaN 
null_count = df['text'].isnull().sum()
print(f"\n1. Kiểm tra null values: {null_count}")

if null_count > 0:
    print(f"   ⚠️ Tìm thấy {null_count} null values!")
    print(f"   🔧 Đang loại bỏ...")
    # Lọc TRỰC TIẾP trên df
    df = df[df['text'].notna()].copy()
    print(f"   ✅ Đã loại bỏ. Còn lại: {len(df):,} documents")

# 2. Kiểm tra empty strings (từ các bước xử lý trước đó)
empty_count = (df['text'].str.strip() == '').sum()
print(f"\n2. Kiểm tra empty strings: {empty_count}")

if empty_count > 0:
    print(f"   ⚠️ Tìm thấy {empty_count} empty strings!")
    print(f"   🔧 Đang loại bỏ...")
    # Lọc TRỰC TIẾP trên df
    valid_idx = df['text'].str.strip() != ''
    df = df[valid_idx].copy()
    print(f"   ✅ Đã loại bỏ. Còn lại: {len(df):,} documents")

# 3. Kiểm tra độ dài tối thiểu 
# Đặt một ngưỡng tối thiểu, ví dụ 5 ký tự
MIN_LEN_FINAL = 5
short_count = (df['text'].str.len() < MIN_LEN_FINAL).sum()
print(f"\n3. Kiểm tra độ dài < {MIN_LEN_FINAL} ký tự: {short_count}")

if short_count > 0:
    print(f"   ⚠️ Có {short_count} comments < {MIN_LEN_FINAL} ký tự")
    print(f"   🔧 Đang loại bỏ...")
    # Lọc TRỰC TIẾP trên df
    valid_idx = df['text'].str.len() >= MIN_LEN_FINAL
    df = df[valid_idx].copy()
    print(f"   ✅ Đã loại bỏ. Còn lại: {len(df):,} documents")

# Reset index một lần DUY NHẤT sau khi đã lọc xong
df = df.reset_index(drop=True)

# Gán X_text và y TỪ df đã sạch
X_text = df['text']
y = df['label']

print(f"\n✅ Dữ liệu đã sạch!")
print(f"   • X_text: {len(X_text):,} documents")
print(f"   • y: {len(y):,} labels")
print(f"   • Toxic: {(y==1).sum():,} ({(y==1).mean()*100:.1f}%)")
print(f"   • Non-toxic: {(y==0).sum():,} ({(y==0).mean()*100:.1f}%)")

KIỂM TRA VÀ XỬ LÝ DỮ LIỆU (ĐÃ SỬA LỖI)
Tổng số documents ban đầu: 164,064

1. Kiểm tra null values: 0

2. Kiểm tra empty strings: 2
   ⚠️ Tìm thấy 2 empty strings!
   🔧 Đang loại bỏ...
   ✅ Đã loại bỏ. Còn lại: 164,062 documents

3. Kiểm tra độ dài < 5 ký tự: 0

✅ Dữ liệu đã sạch!
   • X_text: 164,062 documents
   • y: 164,062 labels
   • Toxic: 16,748 (10.2%)
   • Non-toxic: 147,314 (89.8%)


In [28]:
count_null = df['text'].isnull().sum()
print(count_null)


0


In [29]:
# Lưu dữ liệu đã làm sạch
output_file = '../data/cleaned_dataset.csv'
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✅ Đã lưu vào: {output_file}")

✅ Đã lưu vào: ../data/cleaned_dataset.csv


In [30]:
# Load lại dữ liệu gốc để so sánh
df_original = pd.read_csv('../data/merged_dataset.csv')
df_cleaned = pd.read_csv('../data/cleaned_dataset.csv')

print("="*80)
print("SO SÁNH TRƯỚC VÀ SAU XỬ LÝ")
print("="*80)

print(f"\n📊 Thống kê:")
print(f"   • Tổng số records gốc: {len(df_original):,}")
print(f"   • Tổng số records sau xử lý: {len(df_cleaned):,}")
print(f"   • Số records bị loại: {len(df_original) - len(df_cleaned):,}")
print(f"   • Tiếng Việt trong cleaned: {len(df_cleaned[df_cleaned['language'] == 'vi']):,}")

# Tìm các ví dụ có sự thay đổi rõ rệt
print(f"\n🔍 TÌM CÁC MẪU CÓ THAY ĐỔI LỚN:")
print("="*80)

# Lọc dữ liệu tiếng Việt từ cả 2 dataset
vi_original = df_original[df_original['language'] == 'vi'].reset_index(drop=True)
vi_cleaned = df_cleaned[df_cleaned['language'] == 'vi'].reset_index(drop=True)

print(f"\nTổng số mẫu tiếng Việt: {len(vi_cleaned):,}")

# So sánh 10 mẫu ngẫu nhiên
samples = vi_cleaned.sample(10, random_state=42)

changes_found = 0
for i, (idx, row) in enumerate(samples.iterrows(), 1):
    cleaned_text = row['text']
    
    # Tìm text gốc tương ứng (cùng index sau reset)
    if idx < len(vi_original):
        original_text = vi_original.loc[idx, 'text']
        
        # Chỉ hiển thị nếu có thay đổi
        if original_text.lower().strip() != cleaned_text.strip():
            changes_found += 1
            label = row['label']
            source = row['source']
            
            print(f"\n📝 Mẫu {changes_found}:")
            print(f"   Label: {'🔴 Toxic' if label == 1 else '🟢 Non-toxic'}")
            print(f"   Source: {source}")
            print(f"\n   📥 TRƯỚC ({len(original_text)} ký tự):")
            if len(original_text) > 150:
                print(f"      {original_text[:150]}...")
            else:
                print(f"      {original_text}")
            
            print(f"\n   📤 SAU ({len(cleaned_text)} ký tự):")
            if len(cleaned_text) > 150:
                print(f"      {cleaned_text[:150]}...")
            else:
                print(f"      {cleaned_text}")
            
            # Phân tích thay đổi
            changes = []
            if original_text.lower() != original_text:
                changes.append("✓ Lowercase")
            if '@' in original_text and '@' not in cleaned_text:
                changes.append("✓ Loại @mentions")
            if 'http' in original_text and 'http' not in cleaned_text:
                changes.append("✓ Loại URLs")
            if '_' in cleaned_text:
                changes.append("✓ Expand teencode")
            if len(original_text) != len(cleaned_text):
                changes.append(f"✓ Độ dài: {len(original_text)} → {len(cleaned_text)}")
            
            if changes:
                print(f"\n   🔧 Các xử lý đã áp dụng:")
                for change in changes:
                    print(f"      {change}")
            
            print("-" * 80)

if changes_found == 0:
    print("\n⚠️  Không tìm thấy thay đổi rõ rệt trong 10 mẫu ngẫu nhiên.")
    print("   Có thể do: index không tương ứng hoặc các thay đổi nhỏ")

# Phân tích tổng quan
print("\n📈 PHÂN TÍCH TỔNG QUAN:")
print("="*80)

sample_texts = df_cleaned[df_cleaned['language'] == 'vi']['text'].head(100)

patterns_found = {
    'Có teencode expanded (chứa _)': sample_texts.str.contains('_', na=False).sum(),
    'Toàn lowercase': sample_texts.str.islower().sum(),
    'Có ký tự lặp ≥3 lần': sample_texts.str.contains(r'(.)\1{2,}', na=False).sum(),
    'Trung bình độ dài': f"{sample_texts.str.len().mean():.1f} ký tự"
}

for key, value in patterns_found.items():
    print(f"   • {key}: {value}")

print("\n✅ Hoàn thành phân tích!")

SO SÁNH TRƯỚC VÀ SAU XỬ LÝ

📊 Thống kê:
   • Tổng số records gốc: 164,069
   • Tổng số records sau xử lý: 164,062
   • Số records bị loại: 7
   • Tiếng Việt trong cleaned: 4,496

🔍 TÌM CÁC MẪU CÓ THAY ĐỔI LỚN:

Tổng số mẫu tiếng Việt: 4,496

📝 Mẫu 1:
   Label: 🟢 Non-toxic
   Source: youtube

   📥 TRƯỚC (27 ký tự):
      Mặt buồn thấy cưng quá hà 😂

   📤 SAU (25 ký tự):
      mặt buồn thấy cưng quá hà

   🔧 Các xử lý đã áp dụng:
      ✓ Lowercase
      ✓ Độ dài: 27 → 25
--------------------------------------------------------------------------------

📝 Mẫu 2:
   Label: 🟢 Non-toxic
   Source: youtube

   📥 TRƯỚC (22 ký tự):
      Kẻ độc hành đi ad ơiii

   📤 SAU (58 ký tự):
      haha bà hiền thẩm mỹ kiểu hênh lắm mới xấu đc được z vậy á

   🔧 Các xử lý đã áp dụng:
      ✓ Lowercase
      ✓ Độ dài: 22 → 58
--------------------------------------------------------------------------------

📝 Mẫu 3:
   Label: 🟢 Non-toxic
   Source: youtube

   📥 TRƯỚC (117 ký tự):
      Đúng là im lặng ngắm 

C:\Users\USER\AppData\Local\Temp\ipykernel_11284\2420621992.py:90: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  'Có ký tự lặp ≥3 lần': sample_texts.str.contains(r'(.)\1{2,}', na=False).sum(),
